In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
trustnews=pd.read_csv('../input/trustnews-website-database/trustnews_data.csv')

In [ ]:
wataneya=pd.read_csv('../input/wataneya-website-dataset/wataneya_data.csv')

In [ ]:
print(len(wataneya))

In [ ]:
data=pd.concat([trustnews,wataneya]).sample(frac=1, random_state=42).reset_index(drop=True)
testdata=pd.read_csv('/kaggle/input/icompass-old-data/oldnews.csv')

In [ ]:
testdata.rename(columns={'texte':'texts'},inplace=True)

In [ ]:
wholedata=pd.concat([data,testdata]).sample(frac=1, random_state=42).reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
data.drop(['Unnamed: 0','index'],axis=1,inplace=True)
testdata.drop(['Unnamed: 0'],axis=1,inplace=True)


In [ ]:
import seaborn as sns
sns.countplot(x='fake', data=data)

Data preprocessing

In [ ]:
# def balanced_batch(data):
#   harmful=data.loc[data["fake"]==1]
#   norm_count=int(len(harmful)*2.5)
#   normal=data.loc[data["fake"]==0].sample(n=norm_count)
#   data=harmful.append(normal).sample(frac=1).reset_index(drop=True)
#   return data

# data=balanced_batch(data)

In [ ]:
!pip install pyarabic
!pip install farasapy
!pip install tashaphyne

In [ ]:
from nltk.corpus import stopwords
from textblob import TextBlob
import re
from tashaphyne.stemming import ArabicLightStemmer
from nltk.stem.isri import ISRIStemmer
import tashaphyne.arabic_const as arabconst 
import pyarabic.araby as araby

In [ ]:
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}
ArListem = ArabicLightStemmer()


In [ ]:
def stem(text):
    zen = TextBlob(text)
    words = zen.words
    cleaned = list()
    for w in words:
        ArListem.light_stem(w)
        cleaned.append(ArListem.get_root())
    return " ".join(cleaned)


In [ ]:
def normalizeArabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

In [ ]:
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])

In [ ]:
def remove_alphas(text):
    # Removes any link in the text
      text = re.sub('_', ' ', text)
      text = re.sub('،', '', text)
      text = re.sub('[A-Za-z0-9]','',text)
      return text

In [ ]:
# data['texts'] = data['texts'].apply(lambda x:remove_alphas(x))
# data['texts'] = data['texts'].apply(lambda x:remove_stop_words(x))
# data['texts'] = data['texts'].apply(lambda x:normalizeArabic(x))
# data['texts'] = data['texts'].apply(lambda x:stem(x))

# testdata['texts'] = testdata['texts'].apply(lambda x:remove_alphas(x))
# testdata['texts'] = testdata['texts'].apply(lambda x:remove_stop_words(x))
# testdata['texts'] = testdata['texts'].apply(lambda x:normalizeArabic(x))
# testdata['texts'] = testdata['texts'].apply(lambda x:stem(x))

In [ ]:
# !git clone https://github.com/aub-mind/arabert

In [ ]:
# from arabert.preprocess import ArabertPreprocessor

# model_name="aubmindlab/bert-base-arabertv01"
# arabert_prep = ArabertPreprocessor(model_name=model_name)

# wholedata['texts'] = wholedata['texts'].apply(lambda x:arabert_prep.preprocess(x))
# testdata['texts'] =testdata['texts'].apply(lambda x:arabert_prep.preprocess(x))

In [ ]:
wholedata['texts'] = wholedata['texts'].apply(lambda x:remove_alphas(x))
wholedata['texts'] = wholedata['texts'].apply(lambda x:remove_stop_words(x))
wholedata['texts'] = wholedata['texts'].apply(lambda x:normalizeArabic(x))
wholedata['texts'] = wholedata['texts'].apply(lambda x:stem(x))

In [ ]:
from sklearn.model_selection import train_test_split
X=wholedata.texts.values
y=wholedata.fake.values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.4,shuffle=True, random_state=42,stratify=y)

# X_train=data.texts.values
# y_train=data.fake.values
# X_test=testdata.texts.values
# y_test=testdata.fake.values


In [ ]:
testdata.head()

In [ ]:
print(len(X_test),len(X_train))

In [ ]:
#transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel,AutoModel

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('UBC-NLP/MARBERT')


In [ ]:
MAX_LEN=128

In [ ]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            pad_to_max_length = True,
            return_attention_mask=True,
            truncation=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [ ]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
# val_input_ids, val_attention_masks = tokenize(X_val, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [ ]:
bert_model = TFBertModel.from_pretrained('UBC-NLP/MARBERT')


In [ ]:
def create_model(bert_model,learning,max_len=MAX_LEN):
    
    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=learning)
    loss = 'binary_crossentropy'
    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    embeddings = bert_model([input_ids,attention_masks])[0]
    x=tf.keras.layers.GlobalAveragePooling1D()(embeddings)
    x=tf.keras.layers.Dropout(0.1)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    model.compile(opt, loss=loss, metrics=['accuracy'])
    return model

In [ ]:
model = create_model(bert_model,1e-5,MAX_LEN)
model.summary()

In [ ]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_split=0.2, epochs=1, batch_size=32)

In [ ]:
result_bert = model.predict([test_input_ids,test_attention_masks])


In [ ]:
preds = np.where(result_bert> 0.5, 1, 0)

In [ ]:
def conf_matrix(y, y_pred, title):
    fig, ax =plt.subplots(figsize=(5,5))
    labels=['Negative', 'Positive']
    ax=sns.heatmap(confusion_matrix(y, y_pred), annot=True, cmap="Blues", fmt='g', cbar=False, annot_kws={"size":25})
    plt.title(title, fontsize=20)
    ax.xaxis.set_ticklabels(labels, fontsize=17) 
    ax.yaxis.set_ticklabels(labels, fontsize=17)
    ax.set_ylabel('Test', fontsize=20)
    ax.set_xlabel('Predicted', fontsize=20)
    plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
conf_matrix(y_test, preds,'Bert case arabic check worthiness detection')

In [ ]:
from sklearn.metrics import accuracy_score
accuracy= accuracy_score(y_test, preds)
print("Accuracy :%5f" % (accuracy))

In [ ]:
from sklearn.metrics import f1_score,precision_score,recall_score
fscore=f1_score(y_test,preds, average='binary')
pscore=precision_score(y_test,preds, average='binary')
rscore=recall_score(y_test,preds, average='binary')
print("acc: %.3f" % (accuracy))
print(f"f1_score is :{fscore:.3f}")
print(f"precision is :{pscore:.3f}")
print(f"recall is :{rscore:.3f}")

In [ ]:
# history_dict = history_bert.history
# print(history_dict.keys())

# acc = history_dict['accuracy']
# val_acc = history_dict['val_accuracy']
# loss = history_dict['loss']
# val_loss = history_dict['val_loss']

# epochs = range(1, len(acc) + 1)
# fig = plt.figure(figsize=(10, 6))
# fig.tight_layout()

# plt.subplot(2, 1, 1)
# # r is for "solid red line"
# plt.plot(epochs, loss, 'r', label='Training loss')
# # b is for "solid blue line"
# plt.plot(epochs, val_loss, 'b', label='Validation loss')
# plt.title('Training and validation loss')
# # plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(2, 1, 2)
# plt.plot(epochs, acc, 'r', label='Training acc')
# plt.plot(epochs, val_acc, 'b', label='Validation acc')
# plt.title('Training and validation accuracy')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend(loc='lower right')